In [23]:
import matlab.engine
import os
import numpy as np
from tqdm import trange
import pickle

In [24]:
import pprint 
pp = pprint.PrettyPrinter(indent=4)

In [2]:
test_cases = [
#     'pglib_opf_case24_ieee_rts.m', 
#     'pglib_opf_case73_ieee_rts.m',
    'pglib_opf_case162_ieee_dtc.m',
    'pglib_opf_case300_ieee.m',
]

In [3]:
def get_Pb_lim(Pg_lim, GB_map):
    Pb_lim = {}

    for g_idx in range(len(GB_map)):
        b_idx = GB_map[g_idx]
        try:
            Pb_lim[b_idx][0] = Pb_lim[b_idx][0] + Pg_lim[g_idx][0]
            Pb_lim[b_idx][1] = Pb_lim[b_idx][1] + Pg_lim[g_idx][1]
        except KeyError:
            Pb_lim[b_idx] = np.zeros(2)
            Pb_lim[b_idx][0] = Pg_lim[g_idx][0]
            Pb_lim[b_idx][1] = Pg_lim[g_idx][1]
    return Pb_lim


def get_Pb(Pg, GB_map):
    Pb = {}

    for g_idx in range(len(GB_map)):
        b_idx = GB_map[g_idx]
        try:
            Pb[b_idx] = Pb[b_idx] + Pg[g_idx]
        except KeyError:
            Pb[b_idx] = Pg[g_idx]
    return Pb

In [4]:
def get_Pg_active_constraints(Pg, Pg_lim):
    active_constraints = np.zeros((len(Pg), 2))

    for g_idx in range(Pg_lim.shape[0]):
        if Pg[g_idx] >= Pg_lim[g_idx][0]:
            active_constraints[g_idx][0] = 1
        if Pg[g_idx] <= Pg_lim[g_idx][1]:
            active_constraints[g_idx][1] = 1

    return np.sum(active_constraints, axis=1) % 2


def get_Pb_active_constraints(Pg, Pg_lim, GB_map):
    Pb = get_Pb(Pg, GB_map)
    Pb_lim = get_Pb_lim(Pg_lim, GB_map)
    
    Bg_idx = list(Pb.keys())
    
    active_constraints = np.zeros((len(Bg_idx), 2))
    
    for i in range(len(Bg_idx)):
        b_idx = Bg_idx[i]
        if Pb[b_idx] >= Pb_lim[b_idx][0]:
            active_constraints[i][0] = 1
        if Pb[b_idx] <= Pb_lim[b_idx][1]:
            active_constraints[i][1] = 1
    
    return np.sum(active_constraints, axis=1) % 2


def get_F_active_constraints(F, F_lim):
    active_constraints = []

    for f, f_lim in zip(F, F_lim):
        if abs(f) >= abs(f_lim):
            active_constraints.append(1)
        else:
            active_constraints.append(0)

    return np.array(active_constraints)


def merge_active_constraints(Pg_active, Pb_active, F_active):
    return np.array(list(Pg_active) + list(Pb_active) + list(F_active))

In [5]:
def create_dataset(file_name, dataset_size, std_scaler=0.03):
    print("> creating dataset with {}".format(file_name))

    x = []
    y = []

    org_dir = os.getcwd()
    os.chdir('./matpower7.0/')

    eng = matlab.engine.start_matlab()

    data = eng.dc_opf_solver(file_name, 0.03)

    Pg = np.squeeze(np.array(data['Pg']))
    B_idx = np.squeeze(np.array(data['B_idx']))
    GB_map = np.squeeze(np.array(data['GB_map'])).astype(int)
    F = np.squeeze(np.array(data['F']))
    Pg_lim = np.array(data['Pg_lim'])
    F_lim = np.array(data['F_lim'])
    w = np.squeeze(np.array(data['w']))

    for i in trange(dataset_size):
        data = eng.dc_opf_solver(file_name, std_scaler)
        # assing x data
        x.append(np.array(data['w']))
        # assgin y data
        Pg_active = get_Pg_active_constraints(Pg, Pg_lim)
        Pb_active = get_Pb_active_constraints(Pg, Pg_lim, GB_map)
        F_active = get_F_active_constraints(F, F_lim)
        active_constraints = merge_active_constraints(Pg_active, Pb_active,
                                                      F_active)
        y.append(np.array(active_constraints))

    eng.quit()

    os.chdir(org_dir)

    return {'x': np.squeeze(np.array(x)), 'y': np.array(y)}

## Example of parsing active constraints: DCOPF

In [6]:
org_dir = os.getcwd()
os.chdir('./matpower7.0/')

In [7]:
eng = matlab.engine.start_matlab()

In [8]:
test_case_idx = 0
case_name = test_cases[test_case_idx]

In [9]:
data = eng.dc_opf_solver(case_name, 0.03)

In [10]:
Pg = np.squeeze(np.array(data['Pg']))
B_idx = np.squeeze(np.array(data['B_idx']))
GB_map = np.squeeze(np.array(data['GB_map'])).astype(int)
F = np.squeeze(np.array(data['F']))
Pg_lim = np.array(data['Pg_lim'])
F_lim = np.array(data['F_lim'])
w = np.squeeze(np.array(data['w']))

### Generator active constraints: [Pg_min, Pg_max]]

In [11]:
Pg

array([ 790.33387052,    0.        , 1127.        ,  150.        ,
          0.        ,  600.        ,  150.        ,   38.7182289 ,
        700.        , 2401.99805703,  578.        ,  710.        ])

In [12]:
Pg_lim

array([[1147.,    0.],
       [ 451.,    0.],
       [1127.,    0.],
       [ 366.,    0.],
       [ 110.,    0.],
       [1119.,    0.],
       [ 308.,    0.],
       [ 455.,    0.],
       [ 700.,    0.],
       [2526.,    0.],
       [1593.,    0.],
       [1130.,    0.]])

In [13]:
Pg_active = get_Pg_active_constraints(Pg, Pg_lim)
Pg_active

array([0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.])

### Bus active constraints, due to precense of co-related generation: [Pb_min, Pb_max]

In [14]:
Pb = get_Pb(Pg, GB_map)
Pb

{6: 790.3338705160185,
 73: 0.0,
 76: 1127.0,
 99: 149.99999999999997,
 101: 0.0,
 108: 600.000000000001,
 114: 150.00000000000006,
 118: 38.71822889945886,
 121: 700.0,
 125: 2401.998057032505,
 130: 578.0,
 131: 709.9999999999997}

In [15]:
Pb_lim = get_Pb_lim(Pg_lim, GB_map)
Pb_lim

{6: array([1147.,    0.]),
 73: array([451.,   0.]),
 76: array([1127.,    0.]),
 99: array([366.,   0.]),
 101: array([110.,   0.]),
 108: array([1119.,    0.]),
 114: array([308.,   0.]),
 118: array([455.,   0.]),
 121: array([700.,   0.]),
 125: array([2526.,    0.]),
 130: array([1593.,    0.]),
 131: array([1130.,    0.])}

In [16]:
Pb_active = get_Pb_active_constraints(Pg, Pg_lim, GB_map)
Pb_active

array([0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.])

### Flow Constraints: [-F_max, F_max] 

In [17]:
F

array([ 2.71583105e+02,  4.43649858e+02,  2.88539299e+02, -2.13438391e+02,
       -7.90333871e+02,  5.52087133e+01,  5.40253970e+01, -4.14194977e+00,
        7.43629671e+01,  2.99156882e+00,  1.47963956e+01, -1.59381476e+00,
       -3.78385088e+01, -2.68113787e+02,  2.60913162e+02,  2.95739923e+02,
        2.66670572e+02,  2.29891037e+02, -7.10000000e+02,  3.36690537e+02,
       -2.81481824e+02,  6.77639856e+00, -1.08068815e+01, -9.70370208e+00,
       -1.35046107e+01, -7.54181366e+00, -2.24379174e+01, -2.81481824e+02,
       -1.49232418e+02, -1.91238432e+01, -2.90067157e+01, -3.08384165e+00,
       -8.29828426e+01,  9.85443950e+01, -6.46456229e+01,  3.03786647e+01,
       -1.62348995e+02, -1.28209630e+01, -7.36104284e+00, -3.70367414e+00,
       -2.63569652e+01, -1.60826672e+02, -2.02727436e+00, -1.80581259e+00,
       -2.99384022e+01,  1.08766967e+02,  9.92370572e+00, -1.89311973e+02,
        6.44834520e+00, -9.83378242e-01,  6.39450282e+01,  1.79381154e+00,
       -1.03893687e+01, -

In [18]:
F_lim

array([[ 613.],
       [ 895.],
       [ 470.],
       [ 663.],
       [ 900.],
       [ 605.],
       [ 610.],
       [  29.],
       [ 169.],
       [  17.],
       [  18.],
       [  46.],
       [ 257.],
       [ 514.],
       [ 500.],
       [ 591.],
       [ 644.],
       [ 702.],
       [ 710.],
       [ 672.],
       [ 637.],
       [  26.],
       [ 159.],
       [  60.],
       [  74.],
       [  17.],
       [ 102.],
       [ 684.],
       [ 366.],
       [  47.],
       [  42.],
       [  24.],
       [ 162.],
       [ 238.],
       [ 271.],
       [ 350.],
       [ 336.],
       [  89.],
       [  45.],
       [  18.],
       [  91.],
       [ 240.],
       [ 264.],
       [ 235.],
       [ 244.],
       [ 344.],
       [  18.],
       [ 359.],
       [  21.],
       [  19.],
       [ 184.],
       [  33.],
       [  56.],
       [  55.],
       [ 209.],
       [  37.],
       [ 109.],
       [ 111.],
       [ 220.],
       [ 219.],
       [ 225.],
       [  16.],
       [

In [19]:
F_active = get_F_active_constraints(F, F_lim)
F_active

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Merge all active constriants

In [20]:
active_aconstraints = merge_active_constraints(Pg_active, Pb_active, F_active)
active_aconstraints

array([0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0.

## (Optional) Example of parsing active constraints: ACOPF

In [21]:
data = eng.ac_opf_solver(case_name, 0.03)

In [3]:
pp.pprint(data)

NameError: name 'pp' is not defined

In [22]:
eng.quit()

In [23]:
os.chdir(org_dir)

## Create Datasets for NNs

### example of creating a dataset
- **x**: uncertainty realization as a feature input
- **y**: active constraints index as a label output

In [24]:
dataset = create_dataset(case_name, dataset_size=1)
dataset

> creating dataset with pglib_opf_case162_ieee_dtc.m


100%|██████████| 1/1 [00:00<00:00,  1.72it/s]


{'x': array([ 0.00000000e+00,  0.00000000e+00,  1.83760233e+01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  9.44950778e+00,
         0.00000000e+00, -1.57964993e+01,  0.00000000e+00,  1.93102772e+00,
         2.39508406e+00,  5.16269576e+00, -3.12683168e-01,  2.98678470e-01,
        -1.66415479e+00,  8.89606303e-01, -2.63079372e+00,  5.17368606e-01,
        -1.77719744e+00, -1.74710516e-01,  1.05305227e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  6.41659133e+00, -7.83236353e-02,
        -1.65801363e-01, -6.60652879e-01, -6.59259076e-01,  3.64351322e-02,
         6.95035612e-02,  3.51407111e-01,  2.49569070e+00,  4.47677561e-01,
         0.00000000e+00,  2.76834290e-01,  0.00000000e+00, -1.63362323e+00,
         1.11628487e+00,  0.00000000e+00,  1.68292803e-01,  2.52264606e-01,
         1.57000638e-01, -1.84465307e+00, -2.34740275e-02, -1.47924134e-01,
        -1.08849528e-01,  5.03117185e+00,  0.00000000e+00, -3.16705332e+00,
       

### Build datasets

In [25]:
def save_dataset(test_case, dataset):
    file_name = test_case.split('.')[0]
    file_path = './datasets/'
    file_dir = file_path + file_name + '.pickle'
    outfile = open(file_dir,'wb')
    pickle.dump(dataset, outfile)
    outfile.close()

    
def build_datasets(test_cases, dataset_size):
    for test_case in test_cases:
        # create a dataset
        dataset = create_dataset(test_case, dataset_size)
        # save the dataset
        save_dataset(test_case, dataset)

In [26]:
# dataset_size = 2
dataset_size = 50000

In [27]:
build_datasets(test_cases, dataset_size)

> creating dataset with pglib_opf_case162_ieee_dtc.m


100%|██████████| 50000/50000 [3:14:59<00:00,  4.27it/s]  


> creating dataset with pglib_opf_case300_ieee.m


100%|██████████| 50000/50000 [4:52:27<00:00,  2.85it/s]  


### Check datasets

In [28]:
infile = open('./datasets/pglib_opf_case24_ieee_rts.pickle','rb')
dataset = pickle.load(infile)
infile.close()

In [29]:
dataset

{'x': array([[ 1.58401582,  3.01095666,  3.92517972, ...,  0.        ,
          0.        ,  0.        ],
        [ 3.54218077,  3.2279853 , -4.66372524, ...,  0.        ,
          0.        ,  0.        ],
        [-0.62343597,  2.58585634, -4.13018588, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-1.73786221, -1.83840692,  6.28492785, ...,  0.        ,
          0.        ,  0.        ],
        [-2.94739091, -1.37913875, -7.33716418, ...,  0.        ,
          0.        ,  0.        ],
        [-1.56552295,  3.44187377,  8.9611732 , ...,  0.        ,
          0.        ,  0.        ]]),
 'y': array([[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]])}